In [ ]:
from dataclasses import dataclass
import itertools
import re

In [ ]:
@dataclass
class Vector:
    x: int = 0
    y: int = 0
    z: int = 0

@dataclass
class Moon:
    pos: Vector
    vel: Vector
    
    def __energy(self, attrib):
        return sum([abs(getattr(attrib, axis)) for axis in 'xyz'])
    
    def energy(self):
        return self.__energy(self.pos) * self.__energy(self.vel)

In [ ]:
def parse_input(filename):
    moons = []
    with open(filename) as file:
        for line in file:
            moon = Moon(pos=Vector(*map(int, re.findall(r"=(-?\d+)", line))),
                        vel=Vector())
            moons.append(moon)
    return moons

In [ ]:
def apply_gravity(moons):
    for i, (m1, m2) in enumerate(itertools.combinations(moons, 2)):
        for axis in 'xyz':
            if getattr(m1.pos, axis) > getattr(m2.pos, axis):
                setattr(m1.vel, axis, getattr(m1.vel, axis) - 1)
                setattr(m2.vel, axis, getattr(m2.vel, axis) + 1)
            elif getattr(m1.pos, axis) < getattr(m2.pos, axis):
                setattr(m1.vel, axis, getattr(m1.vel, axis) + 1)
                setattr(m2.vel, axis, getattr(m2.vel, axis) - 1)

In [ ]:
def apply_velocity(moons):
    for moon in moons:
        for axis in 'xyz':
            setattr(moon.pos, axis, getattr(moon.pos, axis) + getattr(moon.vel, axis))

# Part 1

In [ ]:
def part1(filename, steps=1000):
    moons = parse_input(filename)
    
    for i in range(steps):
        apply_gravity(moons)
        apply_velocity(moons)
        
    return sum([moon.energy() for moon in moons])

In [ ]:
# Tests
assert part1("day12-test1.input", 10) == 179
assert part1("day12-test2.input", 100) == 1940

In [ ]:
part1("day12.input")

# Part 2

Let S be the set of all states, and F: S -> S be the mapping from one state of the moons to the next (working as described in the problem statement). Notice that F is a bijection since we can easily calculate the inverse (the previous state from a state). Suppose F has a cycle (or the problem would not be solvable). Then the first repeating state must be the initial state, otherwise, F would not be one-to-one. Hence, F is periodic.

The key is to notice that we can split F into axis components Fx, Fy, Fz, since a state of an axis is independent of states of all the other axes. Then the period length of F is the lowest common multiple of the period lengths of Fx, Fy, and Fz. So we just have to find independently the periods of Fx, Fy, and Fz which are hopefully much shorter than the period of F, and indeed they are shorter.

In [ ]:
from collections import defaultdict
import math
import functools

In [ ]:
def get_state(moons):
    """Returns all values for the x-, y- and z-axis separately"""
    state = defaultdict(list)
    for axis in 'xyz':
        for moon in moons:
            state[axis].append(getattr(moon.pos, axis))
            state[axis].append(getattr(moon.vel, axis))
    return state

In [ ]:
def lcm(a, b):
    "Returns the least common multiple of two numbers"
    return a * b // math.gcd(a, b)

In [ ]:
def part2(filename):
    moons = parse_input(filename)
    initial_state = get_state(moons)

    cycle_size = dict(zip('xyz', [False]*3))
    for step in itertools.count(1):
        apply_gravity(moons)
        apply_velocity(moons)
        state = get_state(moons)
        for axis in state:
            if not cycle_size[axis] and state[axis] == initial_state[axis]:
                print("{}-axis has cycled: {}".format(axis, step))
                cycle_size[axis] = step
        if all(cycle_size.values()):
            break

    return functools.reduce(lcm, cycle_size.values())

In [ ]:
assert part2("day12-test1.input") == 2772
assert part2("day12-test2.input") == 4686774924

In [ ]:
part2("day12.input")